## MPC Tensor - Party 1 - Data Scientist

#### 0. Import Libraries

In [1]:
import torch 
import syft as sy 


#### 1.1 Launch DS Duet server

In [2]:
#1.1 - Launch a Duet Server
duet_p1 = sy.launch_duet(loopback=True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...

♫♫♫ > CONNECTED!



#### 1.2 Join to DO Duet server

In [3]:
duet_p2 = sy.join_duet(loopback=True)

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


In [4]:
duet_p1

<DomainClient: <UID: 68296fc0b2094a8580db0e095dc3a46d>>

## 2 SMPC

### 2.1 Create Session

In [1]:
from sympc.session import Session
from sympc.session import SessionManager

from sympc.tensor import MPCTensor


In [6]:
session = Session(parties=[duet_p1, duet_p2])
print(session)

### 2.2 Send session to all parties

In [7]:
SessionManager.setup_mpc(session)

#### Sum, Substract and Multiply operations

In [8]:
y = torch.Tensor([-5, 0, 1, 2, 3])  # some local data

x_secret = duet_p2.store["#test_tensor_1"]  # secret data to test sum, substract and multiply
x = MPCTensor(secret=x_secret, shape=(1,), session=session)  # MPC Tensor from x_secret
# x.share_ptrs
x

[2022-02-23T15:19:57.286967+0100][CRITICAL][logger]][7360] 'More than one item with tag:#test_tensor_1'


KeyError: 'More than one item with tag:#test_tensor_1'

In [10]:
print(f"X: {x}\n"
      f"Y: {y}")

X: [MPCTensor]
Shape: (1,)
Requires Grad: False
	| <DomainClient: <UID: 1629afe2db6141d9aac944b6aacd62b3>> -> ReplicatedSharedTensorShareTensorUnionPointer
	| <Duet: <UID: 1dd362113c174d25a86e15f07b9d057e>> -> AnyPointer
Y: tensor([-5.,  0.,  1.,  2.,  3.])


In [ ]:
#Private + Public operation
print(f"X + Y = {(x+y).reconstruct()}\n"
      f"X - Y = {(x-y).reconstruct()}\n"
      f"X * Y = {(x*y).reconstruct()}")

[2022-02-23T15:06:40.701376+0100][CRITICAL][logger]][14288] The session b5e960fe-358a-4f08-963c-e3862efcfa8c could not be found
Exception in callback AsyncIOEventEmitter._emit_run.<locals>._callback(<Task finishe...ot be found')>) at C:\Anaconda3\envs\pysyft\lib\site-packages\pyee\_asyncio.py:55
handle: <Handle AsyncIOEventEmitter._emit_run.<locals>._callback(<Task finishe...ot be found')>) at C:\Anaconda3\envs\pysyft\lib\site-packages\pyee\_asyncio.py:55>
Traceback (most recent call last):
  File "C:\Anaconda3\envs\pysyft\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Anaconda3\envs\pysyft\lib\site-packages\pyee\_asyncio.py", line 62, in _callback
    self.emit('error', exc)
  File "C:\Anaconda3\envs\pysyft\lib\site-packages\pyee\_base.py", line 116, in emit
    self._emit_handle_potential_error(event, args[0] if args else None)
  File "C:\Anaconda3\envs\pysyft\lib\site-packages\pyee\_base.py", line 86, in _emit_handle_potential_

#### Matrix multiplication

In [ ]:
x_secret = duet_p2.store["#test_tensor_2"]  # secret data with no access
x_mul = MPCTensor(secret=x_secret, shape=(2,2), session=session)  # MPC Tensor build from x_secret
y_mul = torch.tensor([[5,6],[7,8]])
#y_mul = MPCTensor(secret=x_secret, shape=(2,2), session=session)

In [14]:
print(f"X: {x_mul}\n"
      f"Y: {y_mul}")

X: [MPCTensor]
Shape: (2, 2)
Requires Grad: False
	| <DomainClient: <UID: fa496f8af5dd458fbfd8ad859309f662>> -> ShareTensorPointer
	| <Duet: <UID: e30e0c03f30841a1a40ab7f708235722>> -> ShareTensorPointer
Y: tensor([[5, 6],
        [7, 8]])


In [18]:
print(f"X @ X = \n {(x_mul @ y_mul).reconstruct()}")

X @ X = 
 tensor([[19., 22.],
        [43., 50.]])
